In [ ]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../code'))
import config
from spellcheck import Spellcheck
from util import Util

# hny_pap_nl = pd.read_csv("../data/hny/pap-nl.csv", na_filter=False)
nbo_pap = pd.read_csv("../data/nbo/pap(cap).csv", na_filter=False)
corpus = Util.attachType(nbo_pap, "pap-simple")
corpus = corpus[corpus["type"]=="word"]
words = corpus["pap-simple"].values
spell = Spellcheck(spellchecker_corpus=words)

sentence = "mi man no ta bon pero manan mi kuna ta trese anto bini bek"
big_sentence = ". Presidente di Banko Sentral di Kòrsou i St. Maarten sr. Richard Doornbosch... WILLEMSTAD.- Buskando solushon fo’i 2013 te ku 2019 pa Girobank, a hasi investigashon di e banko. A aserka APC i IIG TOF Holding N.V. (ámbos ta doño di Girobank ku respektivamente 42% i 58%) pa un plan di restrukturashon i re-kapitalisashon pa tin sufisiente kapital den e banko; identifiká potensial kumpradónan ku ke a invertí den e banko. Esaki no tabata algu fásil. Tabatin un défisit"

In [ ]:
import random
corpus_len = nbo_pap.copy()
corpus_len["len"] = nbo_pap.apply(lambda row: len(row["pap-simple"]), axis=1)


random.seed(10)
def screwWord(word, screws=1):
    screwed = word
    
    word_length = len(word)
    for i in range(0, word_length):
        if i < screws:
            screwed = screwed[:i] + screwed[random.randrange(0, word_length)] + screwed[i+1:]
    return screwed


# for word in four_raw:
corpus_len["w"] = corpus_len.apply(lambda row: screwWord(row["pap-simple"]), axis=1)
four_raw = corpus_len[corpus_len.len == 4]
test_four_original = four_raw.head(10)["pap-simple"].values
test_four = four_raw.head(10)["w"].values
print(test_four, test_four_original)

In [ ]:
# Translate.getWordCorrections(sentence, corpus)

In [ ]:
# trans.getFastWordCorrections(sentence, check_alternatives=True)

In [ ]:
import nltk
# trans.getMixedWordCorrections(sentence, corpus)
smarter = "iasabra el a bisami ekos kune i use"
# input_words = Util.findWords(big_sentence)
input_words = test_four
# print(input_words)

In [ ]:
spell.getMixedWordCorrections(input_words, corpus, "Levenstein")

In [ ]:
spell.getMixedWordCorrections(input_words, corpus, metric="Hamming")

In [ ]:
spell.getWordCorrections(input_words, metric="Levenstein")

In [ ]:
spell.getPreSufCorrections(input_words)

In [ ]:
def extremeFind(self, word, d, errors=0):
        found_word = "" 
        found_words = []
        current = self
        # print(f"word:{word} depth:{d}")
        found_key = False

        subs = False
        dels = True
        for i, l in enumerate(word):

            # print(f"index {i}")
            # print(f"checking if {l} has accented")
            if d > 0:
                if l in current.children:
                    current = current.children[l]
                    found_word += l
                    print(f"d > 0 appending letter {l} new found word {found_word}")
                    found_key = True
                else:
                    print(f"d > 0 Didn't find {l}, children are {current.children}")
                    found_key = False
            else:
                print(f"word:{word} depth:{d}")
                # print(f"items are len {len(current.children.keys())}")

                if subs:
                    for key, child in current.children.items():
                        # print(f"checking {accented_l}")
                        # print(f"found {accented_l}")
                        rest_search = word[i+1:]

                        print(f"Checking {key} {rest_search}")
                        found_rest_words = extremeFind(child, rest_search, d=d+1)
                        if found_rest_words:
                            print(f"Word {word}, key {key}, ind {i}, d {d}")
                            print(f"Found {found_rest_words}")
                        if found_rest_words:
                            found_words += [found_word + key + found_rest_word for found_rest_word in found_rest_words]
                            print(f"New found words {found_words}")
                            found_key = True
                    
                if l in current.children:
                    old_root = current
                    current = current.children[l]
                    found_word += l
                    print(f"d:0 Appending letter {l} new found word {found_word}")
                    found_rest_words = extremeFind(old_root, word[i+1:], d=d+1)
                    found_words += [found_rest_word for found_rest_word in found_rest_words]
                    found_key = True
                    if found_rest_words:
                        print(f"Searched {word[i+1:]} and found {found_words}")
                else:
                    if i != len(word) - 1:
                        found_rest_words = extremeFind(current, word[i+1], d=d+1)
                        print(f"{l} was not in children, I searched searched {word[i+1:]} and found {found_words}")
                        return found_rest_words

            print(f"d:{d} {word} letter {l} index {i} Last node? {current.lastNode} found key? {found_key}")
            if not found_key:
                return found_words
            
            if i == len(word)-2 and d==0 and dels:
                if current.lastNode:
                    print("Appending to second to last")
                    found_words.append(found_word)
                    return found_words

            if i == len(word)-1:
                if d == 0:
                    print(f"{word} {i} Last node? {current.lastNode}")
                if current.lastNode:
                    found_words.append(found_word)
                print(f"Returning {found_words}")
                return found_words
                # else:
                #     return ""
def getWordCorrections2(words):
        translations = {}   
        for word in words:
            # Make sure to ignore case if you're making word lowercase
            exists = spell.trie.find(word)
            if not exists:
                all_matches = extremeFind(spell.trie, word, d=0)
                # matches = all_matches[:3]
                # translations
                # print(word, matches)
                # if matches and matches[0][1] > 0:
                if all_matches:
                    # print(word, words_corpus.head(3)["closest"].to_list())
                    translations[word] = list(map(lambda x: x, all_matches))
                # elif not matches:
                #     translations[word] = ["This word is incorrect"]
                
        return translations
# corr = getWordCorrections2(input_words)
# corr = getWordCorrections2(["tesa"])
corr = getWordCorrections2(["mesd", "dmes"])
print(f"Corrections: ", corr)


In [ ]:

def accentFind(self, word):
    found_word = "" 
    found_words = []
    current = self
    print(f"New accent find call for {word}")
    for i, l in enumerate(word):
        found_key = False
        # print(f"Letter:{i}. checking if {l} has accented. Children {}")
        if l in self.accented:
            for accented_l in self.accented[l]:
                # print(f"checking {accented_l}")
                if accented_l in current.children:

                    print(f"found {accented_l}")
                    accented_search = word[i+1:]
                    if i == len(word) - 1:
                        found_accented_words = [""]
                    else:
                        found_accented_words = accentFind(current.children[accented_l], accented_search)
                    print(f"Returning from {accented_search} search with {found_accented_words}")
                    if found_accented_words:
                        found_accented_words = [found_word + accented_l + x for x in found_accented_words]    
                        found_words += found_accented_words
            
        if l in current.children:
            current = current.children[l]
            found_word += l
            found_key = True
            print(f"Search {word}. I found {l} as a key. Total word is {found_word}")

        if not found_key:
            return found_words
        
        if i == len(word)-1:
            if current.lastNode:
                found_words.append(found_word)
            return found_words
accent_words = ["paga", "post", "korsou", "minut", "don'texist", "kòrsou", "korsòu", "bon", "no"]            
accentFind(spell.trie, accent_words[8])

In [ ]:
# characters_test = "(whatever)i don't know, but random. 15 a)e right {,age}()((er)"
characters_test = "min sa 42% 32 fo'i ki dia re-kapitulashon M'a mire n' ta nothing' bai fo'i 'esaki tambe' 'gani sa?' e"
# characters_test = "Buskando solushon fo’i 2013 te ku 2019"
import re
def getWords(sentence):
    # words = re.findall(r'^(.*?)[;\.,!\(].*',without_num)        
    # words = re.findall(r'[^\w\d\'](\w+)[^\w\d\']',sentence) 
    # words = re.findall(r'\b([^\d\W]+[\'-]?[^\d\W\']*)\b',sentence) 
    words = re.findall(r'\b([^\d\W]+[\'\’-]*[^\d\W\']*)',sentence) 
    # words = re.findall(r'\b([\d\w\'-]+)\b',sentence) 
    # words = re.findall(r'([a-z]+[\'-]?(?![\w\'-])[a-z]*)',sentence) 
    return words
getWords(characters_test)
# getWords(big_sentence)